# Indicadores IPCA por UF

## Preparação / Instalação de bibliotecas

In [ ]:
!pip install selenium bs4
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time as t
import datetime as dt
import pytz
from dateutil.relativedelta import relativedelta

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

## Start de Script

### Web Scraping

In [ ]:
url = 'https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?edicao=30452&t=resultados'

In [ ]:
def getpageinfo(url):
  wd = webdriver.Chrome('chromedriver', options=options)
  t.sleep(5)
  wd.get(url)
  t.sleep(5)
  return wd.page_source

### Tratamento dos dados

In [ ]:
def tratamento_ipca():
  tz = pytz.timezone('America/Sao_Paulo')
  dta = dt.datetime.now(tz=tz)

  df = pd.read_html(getpageinfo(url),header=1)
  df[1].rename(columns={'Unnamed: 1':'sub_indice_IPCA'},inplace=True)
  tabela_01_mes = pd.DataFrame(df[1].iloc[:10,1:19])
  tabela_02_mes = pd.pivot_table(tabela_01_mes,columns=['sub_indice_IPCA']).reset_index()
  tabela_02_mes.rename(columns={'sub_indice_IPCA':'index','index':'capital'},inplace=True)
  tabela_02_mes.iloc[:,1:] = tabela_02_mes.iloc[:,1:]/100

  data_definitiva = dta - relativedelta(months=1)
  mes, ano, data = data_definitiva.strftime('%B'), data_definitiva.strftime('%Y'), data_definitiva.strftime('%m/01/%Y')
  
  tabela_02_mes['data'], tabela_02_mes['mes'],tabela_02_mes['ano']  = data, mes, ano
  tabela_02_mes[['capital','sigla_uf']] = tabela_02_mes['capital'].str.split('(', expand=True)
  tabela_02_mes['sigla_uf'] = tabela_02_mes['sigla_uf'].str.replace(')','')
  tabela_02_mes = tabela_02_mes[['data',	'mes',	'ano','capital','sigla_uf','1.Alimentação e bebidas',	'2.Habitação',	'3.Artigos de residência',	'4.Vestuário',	'5.Transportes',	'6.Saúde e cuidados pessoais',	'7.Despesas pessoais',	'8.Educação',	'9.Comunicação',	'Índice geral']]

  tabela_02_mes['capital'] = tabela_02_mes['capital'].str.replace('ã','a').str.replace('é','e').str.replace('í','i').str.replace('ó','o').str.replace('â','a')
  tabela_02_mes['sigla_uf'] = tabela_02_mes['sigla_uf'].fillna('BR')
  tabela_02_mes.rename(columns={'1.Alimentação e bebidas':'alimentacao_bebidas_ipca'
                              ,'2.Habitação':'habitacao_ipca'
                              ,'3.Artigos de residência':'artigos_residencia_ipca'
                              ,'4.Vestuário':'vestuario_ipca'
                              ,'5.Transportes':'transportes_ipca'
                              ,'6.Saúde e cuidados pessoais':'saude_cuidado_ipca'
                              ,'7.Despesas pessoais':'despesas_pessoais_ipca'
                              ,'8.Educação':'educacao_ipca'
                              ,'9.Comunicação':'comunicacao_ipca'
                              ,'Índice geral':'indice_geral_ipca'
                              },inplace=True)
  return tabela_02_mes

## Tabela Final

In [ ]:
tabela_ipca = tratamento_ipca()
tabela_ipca

sub_indice_IPCA,data,mes,ano,capital,sigla_uf,alimentacao_bebidas_ipca,habitacao_ipca,artigos_residencia_ipca,vestuário_ipca,transportes_ipca,saude_cuidado_ipca,despesas_pessoais_ipca,educacao_ipca,comunicacao_ipca,indice_geral_ipca
0,04/01/2021,April,2021,Aracaju,SE,0.70,0.93,0.90,-1.89,4.73,0.03,0.10,0.49,-0.17,1.09
1,04/01/2021,April,2021,Belo Horizonte,MG,-0.09,0.83,1.47,-0.14,5.40,0.21,0.01,-0.97,-0.14,1.18
2,04/01/2021,April,2021,Belem,PA,0.30,1.66,1.42,-0.52,2.56,0.03,-0.14,0.23,-0.60,0.80
3,04/01/2021,April,2021,Brasil,BR,0.13,0.81,0.69,0.29,3.81,-0.02,0.04,-0.52,-0.07,0.93
4,04/01/2021,April,2021,Brasilia,DF,1.16,1.16,-0.43,-0.23,4.94,-0.09,0.37,-0.17,-0.29,1.44
5,04/01/2021,April,2021,Campo Grande,MS,-0.96,0.66,0.17,0.95,4.47,0.35,0.34,-0.62,0.22,0.96
6,04/01/2021,April,2021,Curitiba,PR,0.96,0.72,0.87,-0.12,4.19,0.05,-0.29,-0.39,0.00,1.33
7,04/01/2021,April,2021,Fortaleza,CE,0.63,0.09,-0.20,-0.37,2.95,0.15,0.21,0.16,-0.29,0.72
8,04/01/2021,April,2021,Goiania,GO,0.69,-0.34,0.38,-0.33,5.65,-0.28,0.23,0.52,-0.23,1.46
9,04/01/2021,April,2021,Grande Vitoria,ES,-0.51,1.47,0.59,-0.54,3.93,0.17,0.08,0.59,-0.21,1.10
